In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [8]:
# Createa  set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-90.000, high=90.000, size =1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [9]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

600

In [11]:
# Import the datetime module from the datetime library.
from datetime import datetime
import requests
from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [15]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 51 of Set 23 | cape town
Processing Record 52 of Set 23 | illoqqortoormiut
City not found. Skipping...
Processing Record 53 of Set 23 | hermanus
Processing Record 54 of Set 23 | cazaje
City not found. Skipping...
Processing Record 55 of Set 23 | umm durman
City not found. Skipping...
Processing Record 56 of Set 23 | calabozo
Processing Record 57 of Set 23 | ushuaia
Processing Record 58 of Set 23 | kankon
Processing Record 59 of Set 23 | camacha
Processing Record 60 of Set 23 | vertientes
Processing Record 61 of Set 23 | nador
Processing Record 62 of Set 23 | los algarrobos
Processing Record 63 of Set 23 | taolanaro
City not found. Skipping...
Processing Record 64 of Set 23 | grand river south east
City not found. Skipping...
Processing Record 65 of Set 23 | saint george
Processing Record 66 of Set 23 | coihaique
Processing Record 67 of Set 23 | arkhangelsk
Processing Record 68 of Set 23 | chuy
Processing Record 69 of Set 23 | upernavik
Processing Record 70 of Set 23 |

Processing Record 35 of Set 26 | bulembu
Processing Record 36 of Set 26 | benavente
Processing Record 37 of Set 26 | usinsk
Processing Record 38 of Set 26 | thinadhoo
Processing Record 39 of Set 26 | attawapiskat
City not found. Skipping...
Processing Record 40 of Set 26 | maridi
Processing Record 41 of Set 26 | saldanha
Processing Record 42 of Set 26 | marcona
City not found. Skipping...
Processing Record 43 of Set 26 | vetluzhskiy
Processing Record 44 of Set 26 | tsihombe
City not found. Skipping...
Processing Record 45 of Set 26 | adrar
Processing Record 46 of Set 26 | richards bay
Processing Record 47 of Set 26 | los llanos de aridane
Processing Record 48 of Set 26 | abalak
Processing Record 49 of Set 26 | aromashevo
Processing Record 50 of Set 26 | oriximina
Processing Record 1 of Set 27 | warqla
City not found. Skipping...
Processing Record 2 of Set 27 | maniitsoq
Processing Record 3 of Set 27 | barranca
Processing Record 4 of Set 27 | antofagasta
Processing Record 5 of Set 27 | 

Processing Record 16 of Set 30 | villa rica
Processing Record 17 of Set 30 | ndele
Processing Record 18 of Set 30 | point fortin
Processing Record 19 of Set 30 | oranjemund
Processing Record 20 of Set 30 | tigre
Processing Record 21 of Set 30 | porto santo
Processing Record 22 of Set 30 | ushtobe
Processing Record 23 of Set 30 | cordoba
Processing Record 24 of Set 30 | opuwo
Processing Record 25 of Set 30 | edd
Processing Record 26 of Set 30 | cross lanes
Processing Record 27 of Set 30 | mouzakion
City not found. Skipping...
Processing Record 28 of Set 30 | mehamn
Processing Record 29 of Set 30 | jacareacanga
Processing Record 30 of Set 30 | lekoni
Processing Record 31 of Set 30 | kongwa
Processing Record 32 of Set 30 | trelew
Processing Record 33 of Set 30 | srivardhan
Processing Record 34 of Set 30 | axim
Processing Record 35 of Set 30 | kazalinsk
City not found. Skipping...
Processing Record 36 of Set 30 | brunico
Processing Record 37 of Set 30 | alappuzha
Processing Record 38 of Se

Processing Record 6 of Set 34 | mitsamiouli
Processing Record 7 of Set 34 | taguatinga
Processing Record 8 of Set 34 | inyonga
Processing Record 9 of Set 34 | manhush
Processing Record 10 of Set 34 | khanpur
Processing Record 11 of Set 34 | burica
City not found. Skipping...
Processing Record 12 of Set 34 | castelo branco
Processing Record 13 of Set 34 | tupancireta
Processing Record 14 of Set 34 | atar
Processing Record 15 of Set 34 | mahon
Processing Record 16 of Set 34 | shakiso
Processing Record 17 of Set 34 | san-pedro
Processing Record 18 of Set 34 | plouzane
Processing Record 19 of Set 34 | fatehpur
Processing Record 20 of Set 34 | frontignan
Processing Record 21 of Set 34 | copiapo
Processing Record 22 of Set 34 | nova olimpia
Processing Record 23 of Set 34 | uvat
Processing Record 24 of Set 34 | remiremont
Processing Record 25 of Set 34 | caluquembe
Processing Record 26 of Set 34 | gusau
Processing Record 27 of Set 34 | saryshagan
City not found. Skipping...
Processing Record 

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,52.88,98,20,1.99,ZA,2021-06-14 03:34:40
1,Hermanus,-34.4187,19.2345,51.91,84,2,6.51,ZA,2021-06-14 03:39:36
2,Calabozo,8.9242,-67.4293,75.63,78,43,1.95,VE,2021-06-14 03:39:36
3,Ushuaia,-54.8000,-68.3000,37.06,75,75,11.50,AR,2021-06-14 03:36:10
4,Kankon,15.0167,74.0167,80.40,90,100,13.15,IN,2021-06-14 03:39:37
5,Camacha,33.0833,-16.3333,66.96,68,40,12.66,PT,2021-06-14 03:39:37
6,Vertientes,21.2594,-78.1464,77.61,88,98,4.43,CU,2021-06-14 03:39:37
7,Nador,35.1740,-2.9287,69.87,94,0,0.00,MA,2021-06-14 03:39:38
8,Los Algarrobos,8.5000,-82.4333,78.46,96,100,3.98,PA,2021-06-14 03:39:38
9,Saint George,37.1041,-113.5841,97.39,5,1,11.50,US,2021-06-14 03:39:38


In [23]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Cape Town,ZA,2021-06-14 03:34:40,-33.9258,18.4232,52.88,98,20,1.99
1,Hermanus,ZA,2021-06-14 03:39:36,-34.4187,19.2345,51.91,84,2,6.51
2,Calabozo,VE,2021-06-14 03:39:36,8.9242,-67.4293,75.63,78,43,1.95
3,Ushuaia,AR,2021-06-14 03:36:10,-54.8000,-68.3000,37.06,75,75,11.50
4,Kankon,IN,2021-06-14 03:39:37,15.0167,74.0167,80.40,90,100,13.15
...,...,...,...,...,...,...,...,...,...
544,West Bay,KY,2021-06-14 03:42:08,19.3667,-81.4167,83.07,81,20,1.99
545,Lyman,UA,2021-06-14 03:42:09,48.9884,37.8023,65.77,79,61,3.29
546,Abu Samrah,SY,2021-06-14 03:42:09,35.3029,37.1841,70.93,38,0,4.36
547,Seduva,LT,2021-06-14 03:42:09,55.7667,23.7667,48.63,83,4,8.19


In [25]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")